# LOCO-ANS codec configuration generation
This jupyter notebook allows to interactivebly select the coder parameters and generate the configuration file and tANS tables for LOCO-ANS codec

## Publication
This work has associated publications:

### Algorithm:

Title: "LOCO-ANS: An optimization of JPEG-LS using an efficient and low complexity coder based on ANS"

Authors: Tobías Alonso, Gustavo Sutter, and Jorge E. López de Vergara

### Hardware implementation:

Title:

Authors: Tobías Alonso, Gustavo Sutter, and Jorge E. López de Vergara



In [ ]:
import numpy as np
import jpegls_ans
import os

In [ ]:
NOTEBOOK_DIR = os.getcwd()
REPO_DIR = os.path.dirname(NOTEBOOK_DIR)

DEST_FOLDER = REPO_DIR + "/modules/"
DEST_FOLDER_TABLES=DEST_FOLDER+"/ANS_tables"

y_output_encoder_file = DEST_FOLDER_TABLES+ "/tANS_y_encoder_table.dat"  
y_output_decoder_file = DEST_FOLDER_TABLES +"/tANS_y_decoder_table.dat"

z_output_encoder_file = DEST_FOLDER_TABLES+ "/tANS_z_encoder_table.dat"  
z_output_decoder_file = DEST_FOLDER_TABLES +"/tANS_z_decoder_table.dat"

coder_config_file =DEST_FOLDER+"/coder_config.hpp"


# Settings

Choose one of the configurations used in the publication or create a custom one

In [ ]:
max_iters = 7 #INPUT_BPP-1
buffer_size = 2048

In [ ]:
Configuration = "Ntc6_Stfg8_ANS7"

if Configuration == "10_bit_coarse_grain_16bit":
    max_iters = 15
    ANS_STATE_SIZE=10
    # y coding
    Nt_PRECISION = 6
    Half_Bernuulli_coder=False 
    Nt_quantizer_centered=True

    # z coding
    St_PRECISION = 9
    St_extra_precision = 1
    MAX_ST = 3000
    MAX_CARDINALITY=64
elif Configuration == "9_bit_coarse_grain_16bit":
    max_iters = 15
    ANS_STATE_SIZE=9
    # y coding
    Nt_PRECISION = 6
    Half_Bernuulli_coder=False 
    Nt_quantizer_centered=True

    # z coding
    St_PRECISION = 9
    St_extra_precision = 1
    MAX_ST = 2000
    MAX_CARDINALITY=32
elif Configuration == "8_bit_coarse_grain_16bit":
    max_iters = 15
    ANS_STATE_SIZE=8
    # y coding
    Nt_PRECISION = 6
    Half_Bernuulli_coder=False 
    Nt_quantizer_centered=True

    # z coding
    St_PRECISION = 9
    St_extra_precision = 1
    MAX_ST = 1000
    MAX_CARDINALITY=32
elif Configuration == "Ntc6_Stfg8_ANS7":
    ANS_STATE_SIZE=7
    # y coding
    Nt_PRECISION = 6
    Half_Bernuulli_coder=False 
    Nt_quantizer_centered=True

    # z coding
    St_PRECISION = 8
    St_extra_precision = 2
    MAX_ST = 100
    MAX_CARDINALITY=8
elif Configuration == "Ntc6_Stcg8_ANS7":
    ANS_STATE_SIZE=7
    # y coding
    Nt_PRECISION = 6
    Half_Bernuulli_coder=False 
    Nt_quantizer_centered=True

    # z coding
    St_PRECISION = 8
    St_extra_precision = 1
    MAX_ST = 100
    MAX_CARDINALITY=8
elif Configuration == "Ntc6_Stcg7_ANS6":
    ANS_STATE_SIZE=6
    # y coding
    Nt_PRECISION = 6
    Half_Bernuulli_coder=False 
    Nt_quantizer_centered=True

    # z coding
    St_PRECISION = 7
    St_extra_precision = 1
    MAX_ST = 100
    MAX_CARDINALITY=8
elif Configuration == "Ntc5_Stfg6_ANS5":
    ANS_STATE_SIZE=5
    # y coding
    Nt_PRECISION = 5
    Half_Bernuulli_coder=False 
    Nt_quantizer_centered=True

    # z coding
    St_PRECISION = 6
    St_extra_precision = 2
    MAX_ST = 64
    MAX_CARDINALITY=8
elif Configuration == "Ntc5_Stcg6_ANS5":
    ANS_STATE_SIZE=5
    # y coding
    Nt_PRECISION = 5
    Half_Bernuulli_coder=False 
    Nt_quantizer_centered=True

    # z coding
    St_PRECISION = 6
    St_extra_precision = 1
    MAX_ST = 100
    MAX_CARDINALITY=8
elif Configuration == "Ntc4_Stcg5_ANS4":
    ANS_STATE_SIZE=4
    # y coding
    Nt_PRECISION = 4
    Half_Bernuulli_coder=False 
    Nt_quantizer_centered=True

    # z coding
    St_PRECISION = 5
    St_extra_precision = 1
    MAX_ST = 32
    MAX_CARDINALITY=8
else:
    assert False, "Unknown configuration"


## Get sets of reconstruction values

In [ ]:
Max_Nt_idx = (1<<(Nt_PRECISION-1)) -1
p_modes = jpegls_ans.get_p_modes(Nt_PRECISION,Nt_quantizer_centered,Max_Nt_idx,Half_Bernuulli_coder)

if St_extra_precision ==1:
    St_modes = jpegls_ans.get_St_modes_const_ratio(St_PRECISION,MAX_ST)
elif St_extra_precision ==2:
    St_modes = jpegls_ans.get_St_modes_const_ratio_uniform_1(St_PRECISION,MAX_ST)
else:
    assert False, " not supported"

print("Len p_modes:",len(p_modes))
print("St_modes| len:",len(St_modes)," | Min St: %.2e" % St_modes[0]," | Max St (theta): %.2f (%.4f)" % (St_modes[len(St_modes)-1],St_modes[len(St_modes)-1]/(St_modes[len(St_modes)-1]+1)) )

## Get tables to code y variable

In [ ]:
# y coding
# TODO: optimize max_code_len. It can be higher that ANS_STATE_SIZE
y_ANS_encode_tables,y_ANS_decode_tables =  jpegls_ans.get_adaptable_ANS_tables(
                        ANS_address_size = ANS_STATE_SIZE,
                       ec_modes = p_modes, 
                       symbol_source_generator = lambda p : [1-p,p], # p = P(y==1)
                       max_src_card = 2)


## Get tables to code z variable


In [ ]:
# z coding
max_z_coder_card = jpegls_ans.get_max_z_coder_cardinality(St_modes,ANS_STATE_SIZE,MAX_CARDINALITY)
# TODO: optimize max_code_len. I t can be higher that ANS_STATE_SIZE
z_ANS_encode_tables,z_ANS_decode_tables =  jpegls_ans.get_adaptable_ANS_tables(
                            ANS_address_size = ANS_STATE_SIZE,
                           ec_modes = St_modes, 
                           symbol_source_generator = lambda St : jpegls_ans.get_symbol_src_geo(
                                       (St/(St+1)),max_code_len=ANS_STATE_SIZE,max_symbols=max_z_coder_card,
                                       just_pow_of_2 = True,min_symbols=2),
                           max_src_card = max_z_coder_card)


### Get array of source cardinality per theta_q

In [ ]:
cardinality_array_strings = jpegls_ans.get_cardinality_array(St_modes,ANS_STATE_SIZE,max_z_coder_card)

## Create configuration

In [ ]:

config = jpegls_ans.template_config

if St_extra_precision ==1:
    config = config.replace("$CTX_ST_FINER_QUANT$","false" )
elif St_extra_precision ==2:
    config = config.replace("$CTX_ST_FINER_QUANT$","true" )
else:
    assert False, " not supported"

    #define ANS_MAX_SRC_CARDINALITY (1 + 8)
config = config.replace("$CTX_ST_PRECISION$",str(St_PRECISION) )
config = config.replace("$MAX_ST_IDX$", str(len(St_modes)-1))
config = config.replace("$CTX_NT_CENTERED_QUANT$","true" if Nt_quantizer_centered else "false")
config = config.replace("$CTX_NT_PRECISION$",str(Nt_PRECISION))
config = config.replace("$HALF_Y_CODER$","true" if Half_Bernuulli_coder else "false")


# ANS coder 
config = config.replace("$ANS_MAX_SRC_CARDINALITY$",str(MAX_CARDINALITY+1) )
config = config.replace("$EE_MAX_ITERATIONS$",str(max_iters) )
config = config.replace("$EE_BUFFER_SIZE$",str(buffer_size))
config = config.replace("$NUM_ANS_THETA_MODES$",str(len(St_modes)))
config = config.replace("$NUM_ANS_P_MODES$",str(Max_Nt_idx+1) )
config = config.replace("$NUM_ANS_STATES$",str(2**ANS_STATE_SIZE))

config = config.replace("$CARDINALITY_ARRAYS$",cardinality_array_strings)


## Store configuration files

In [ ]:
# y tables
jpegls_ans.store_adaptative_ANS_encoding_tables(y_output_encoder_file,y_ANS_encode_tables,p_modes)    
jpegls_ans.store_adaptative_ANS_decoding_tables(y_output_decoder_file,y_ANS_decode_tables,p_modes)
# z tables
jpegls_ans.store_adaptative_ANS_encoding_tables(z_output_encoder_file,z_ANS_encode_tables,St_modes)
jpegls_ans.store_adaptative_ANS_decoding_tables(z_output_decoder_file,z_ANS_decode_tables,St_modes)

#config file
with open(coder_config_file, "w") as f:
    f.write(config)